## **FACE MASK DETECTION USING VCG19**
The Datasets used are:
1. [Face Mask Detection](https://www.kaggle.com/datasets/andrewmvd/face-mask-detection),uploaded by Larxel on Kaggle.
2. [Face Mask Detection ~12K Images Dataset](https://www.kaggle.com/datasets/ashishjangra27/face-mask-12k-images-dataset) ,uploaded by Ashish Jangra on Kaggle.



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# *Installing Requirements*

In [ ]:
import cv2
from scipy.spatial import distance
import matplotlib.pyplot as plt

from keras.applications.vgg19 import VGG19,preprocess_input
from keras import Sequential
from keras.layers import Flatten,Dense
from keras.preprocessing.image import ImageDataGenerator

# *Loading The Data*

In [ ]:
train_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Train'
test_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Test'
val_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation'

# *Cascade Classifier*
[OpenCV Cascade Classifier Documentation](https://docs.opencv.org/3.4/db/d28/tutorial_cascade_classifier.html)

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def face_detector(face_image_path):
    face_model = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml")
    img = cv2.imread(face_image_path)
    img_gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    img_color = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    faces = face_model.detectMultiScale3(img, minNeighbors=6,outputRejectLevels =True)
    faces_probs = sigmoid(faces[-1])
    for i,d in enumerate(faces[0]):
        (x,y,w,h) = d
        if faces_probs[i]>0.95:
            cv2.rectangle(img_color,(x,y),(x+w,y+h),(0,0,255),1)
    plt.figure(figsize=(10,10))
    plt.imshow(img_color)

    
face_detector("../input/face-mask-detection/images/maksssksksss242.png")

# *Data Agumentation*

In [ ]:
def data_agument(train_dir,val_dir,test_dir,target_size_ = (128,128),zoom_range_ = 0.2,shear_range_=0.2,batch_size_ = 32,class_mode_ = 'binary'):
    train_datagen = ImageDataGenerator(rescale=1.0/255, horizontal_flip=True, zoom_range=zoom_range_,shear_range=shear_range_)
    train_generator = train_datagen.flow_from_directory(directory=train_dir,target_size=target_size_,class_mode='binary',batch_size=batch_size_)

    val_datagen = ImageDataGenerator(rescale=1.0/255)
    val_generator = train_datagen.flow_from_directory(directory=val_dir,target_size=target_size_,class_mode=class_mode_,batch_size=batch_size_)

    test_datagen = ImageDataGenerator(rescale=1.0/255)
    test_generator = train_datagen.flow_from_directory(directory=test_dir,target_size=target_size_,class_mode=class_mode_,batch_size=batch_size_)
    return train_generator,val_generator,test_generator
train_gen,val_gen,test_gen = data_agument(train_dir,val_dir,test_dir)

# *Build Model and Train*

In [ ]:
def build_train_model(train_generator,test_generator,val_generator,epoch = 25):
    vgg19 = VGG19(weights = 'imagenet',include_top = False,input_shape = (128,128,3))
    for layer in vgg19.layers:
        layer.trainable = False

    model = Sequential()
    model.add(vgg19)
    model.add(Flatten())
    model.add(Dense(1,activation = 'sigmoid'))
    model.summary()

    model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

    history = model.fit_generator(generator=train_generator,
                                  steps_per_epoch=len(train_generator)//32,
                                  epochs=epoch,validation_data=val_generator,
                                  validation_steps=len(val_generator)//32)
    model.save('masknet.h5')
    return history,model
history,model = build_train_model(train_gen,test_gen,test_gen)

# *Evaluatation*

In [ ]:
def evaluate(test_generator):
    accuracy = model.evaluate_generator(test_generator)
    print("VGG19 MODEL ACCURACY : "+ str(accuracy[1]))
    return accuracy
accuracy = evaluate(test_gen)    

# *Ploting Loss and Accuracy*

In [ ]:
plt.plot(history.history['loss'])
plt.title('---: Model Loss :---')
plt.ylabel('<-------Loss---------->')
plt.xlabel('<-------Epoch--------->')
plt.legend(['Loss'], loc='lower right')
plt.show()

plt.plot(history.history['accuracy'])
plt.title('---: Model Accuracy :---')
plt.ylabel('<--------Accuracy-------->')
plt.xlabel('<--------Epoch---------->')
plt.legend(['Accuracy'], loc='lower right')
plt.show()

# *Single Image Prediction*

In [ ]:
def predict(sample_img_path):
    mask_label = {0:'MASK',1:'NO MASK'}
    dist_label = {0:(0,255,0),1:(255,0,0)}
    face_model = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")
    img = cv2.imread(sample_img_path)
    img_gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    img_color = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    faces = face_model.detectMultiScale3(img, minNeighbors=6,outputRejectLevels =True)
    print(faces[0])
    print(faces[-1])
    
    faces_probs = sigmoid(faces[-1])
    print(faces_probs)
    for i,d in enumerate(faces[0]):
        (x,y,w,h) = d
        if faces_probs[i]>0.95:
            cv2.rectangle(img_color,(x,y),(x+w,y+h),(0,0,255),4)
    plt.figure(figsize=(10,10))
    plt.imshow(img_color)
    plt.show()
    for i in range(len(faces[0])):
        if faces_probs[i]>0.95:
            (x,y,w,h) = faces[0][i]
            crop = img_color[y:y+h,x:x+w]
            crop = cv2.resize(crop,(128,128))
            crop = np.reshape(crop,[1,128,128,3])/255.0
            mask_result = model.predict(crop)
            #print(type(mask_result))
            #print(mask_result)
            cv2.putText(img_color,mask_label[np.round(mask_result[0][0])] +" "+ str(faces_probs[i]) ,(x, y-10),
            cv2.FONT_HERSHEY_SIMPLEX,0.5,dist_label[np.round(mask_result[0][0])],2)
            cv2.rectangle(img_color,(x,y),(x+w,y+h),dist_label[np.round(mask_result[0][0])],2)
    plt.figure(figsize=(10,10))
    plt.imshow(img_color)
    plt.show()
predict("../input/face-mask-detection/images/maksssksksss243.png")

In [ ]:
print("ACCURACY OF VCG19 MODEL : {} ".format(accuracy[1]))